In [15]:
import pandas as pd
import numpy as np
from itertools import product
import pymysql

In [ ]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [ ]:
# 拉取数据，并分组（描述性，结论性）
def get_and_devide_TagRecode(path):
    dict_ = pd.read_excel(path)
    id_list = dict_.loc[:,'id'].values.tolist()
    tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])
    for items in id_list:
        id_str = "'"+ items + "'"
        tagsql = 'SELECT ORDER_CODE,TAG_ID \
        FROM T66_APOLLO_ANALYSIS_TAG \
        WHERE SOURCE_TYPE = 2 \
        AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
        tag = pd.read_sql(tagsql,con=conn)
        tag_df = pd.concat([tag_df,tag],axis = 0)
    dict_ = thyroid.set_index('tag_id').to_dict()['abbr']
    tag_df.replace(dict_,inplace=True)
    tag_df = tag_df.drop_duplicates()
    DescList = dict_.loc[dict_['logic']== "A",'abbr'].tolist()
    SummList = dict_.loc[dict_['logic']=="a",'abbr'].tolist()
    DescDf = tag_df.loc[tag_df['tag_id'].isin(desclist)]
    SummDf = tag_df.loc[tag_df['tag_id'].isin(summlist)] 
    return DescDf,SummDf

# 穷举订单模式
def ExhaustPatterns(DescDf,SummDf):
    ordercode=[]
    desclist=[]
    summlist=[]
    for item in DescDf['order_code'].drop_duplicates():
        desclistsub=DescDf.loc[DescDf['order_code']== item,'tag_id'].tolist()
        summlistsub=SummDf.loc[SummDf['order_code']== item,'tag_id'].tolist()
        ordercode.append(item)
        desclist.append(desclistsub)
        summlist.append(summlistsub)
    TagOrderCode=pd.DataFrame({'ordercode':ordercode,'desclist':desclist,'summlist':summlist}) 
    return TagOrderCode

# 聚合分析，进行拆分
def Count_and_Disassemble(TagOrderCode,solid_path):
    df = TagOrderCode.applymap(lambda x: tuple(x))
    df_tuple = pd.DataFrame(df.groupby(['desclist','summlist'])['ordercode'].count()).reset_index()
    solid = pd.read_excel(solid_path)
    
    desc_list = []
    summ_list = []
    count_list = []
    source_model = []
    S_NO_desc = []
    S_NO_summ = []
    S_NO_count = []

    for line in df_tuple.values:
        desc = eval(line[0])
        summ = eval(line[1])
        count = line[2]  

        if len(summ)==0:
            source = "summary_missing"
            for index in range(len(desc)):
                descsub = desc[index]
                desc_list.append(descsub)
                summ_list.append(0)
                count_list.append(count)
                source_model.append(source)

        elif (len(desc)== 1 & len(summ)==1):
            source = "complete"
            desc_list.append(desc[0])
            summ_list.append(summ[0])
            count_list.append(count)
            source_model.append(source)
        else :
            source = "split"
            descsub_list = []
            summsub_list = []

            for n in product(desc,summ):
                dfsub = pd.DataFrame(product(desc,summ), columns = ['descsub_list','summsub_list'])
                dfsub_solid_merge = pd.merge(dfsub,solid,how = 'left')
                df_new = dfsub_solid_merge.loc[dfsub_solid_merge.solid.values != 0,["desc","summ"]]
            if dfsub_solid_merge.solid.values.sum()== 0 :
                    S_NO_desc.append(desc)
                    S_NO_summ.append(summ)
                    S_NO_count.append(count)                      
            for index_n, item in enumerate (np.unique(df_new.desc)):
                countsub = count/len(np.unique(df_new.desc))
                summsublist = df_new.loc[df_new["desc"].isin ([item]),'summ']
                for summsub_ in summsublist:
                    countsub_ = countsub/len(summsublist)
                    desc_list.append(item)
                    summ_list.append(summsub_)
                    count_list.append(countsub_)
                    source_model.append(source)
    final = pd.DataFrame({"desc":desc_list,"summ":summ_list,"count":count_list,"source_model":source_model}) 
    NO_ = pd.DataFrame({"desc":S_NO_desc,"summ":S_NO_summ,"count":S_NO_count})
    return final,NO_

if __name__ == '__main__':
    DictPath = ("字典路径")
    SolidPath = ("逻辑路径")
    DescDf,SummDf = get_and_devide_TagRecode（DictPath）
    TagOrderCode = ExhaustPatterns（DescDf,SummDf）
    final,NO = Count_and_Disassemble(TagOrderCode,SolidPath) 

In [ ]:
from apyori import apriori

In [27]:
data_A = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\A.xlsx",index_col = 0)
data_AB = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\AB.xlsx",index_col = 0)
data_abbr = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\abbr.xlsx",index_col = 0)

In [46]:
def AprioriResult(data):
    DataSet = data.taglist.values.tolist()
    result = pd.DataFrame(apriori(transactions= DataSet,min_support = 0.6,min_confidence = 0.7))
    return result

result_A = AprioriResult(data_A)
# result_AB = AprioriResult(data_AB)
# result_abbr = AprioriResult(data_abbr)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [2]:
# apriori其他参数说明:
min_support -- The minimum support of relations (float).最小支持度，可用来筛选项集
min_confidence -- The minimum confidence of relations (float).最小可信度，可用来筛选项集
min_lift -- The minimum lift of relations (float).未知
max_length -- The maximum length of the relation (integer).未知

In [43]:
data = [['豆奶','莴苣'],
        ['莴苣','尿布','葡萄酒','甜菜'],
        ['豆奶','尿布','葡萄酒','橙汁'],
        ['莴苣','豆奶','尿布','葡萄酒'],
        ['莴苣','豆奶','尿布','橙汁']]

result = list(apriori(transactions=data,min_support =0.5))
result_ = list(apriori(transactions=data))

In [41]:
pd.DataFrame(result)

,items,support,ordered_statistics
0,(尿布),0.8,"[((), (尿布), 0.8, 1.0)]"
1,(莴苣),0.8,"[((), (莴苣), 0.8, 1.0)]"
2,(葡萄酒),0.6,"[((), (葡萄酒), 0.6, 1.0)]"
3,(豆奶),0.8,"[((), (豆奶), 0.8, 1.0)]"
4,"(尿布, 莴苣)",0.6,"[((), (尿布, 莴苣), 0.6, 1.0), ((尿布), (莴苣), 0.7499..."
5,"(尿布, 葡萄酒)",0.6,"[((), (尿布, 葡萄酒), 0.6, 1.0), ((尿布), (葡萄酒), 0.74..."
6,"(豆奶, 尿布)",0.6,"[((), (豆奶, 尿布), 0.6, 1.0), ((尿布), (豆奶), 0.7499..."
7,"(豆奶, 莴苣)",0.6,"[((), (豆奶, 莴苣), 0.6, 1.0), ((莴苣), (豆奶), 0.7499..."


In [44]:
pd.DataFrame(result_)

,items,support,ordered_statistics
0,(尿布),0.8,"[((), (尿布), 0.8, 1.0)]"
1,(橙汁),0.4,"[((), (橙汁), 0.4, 1.0)]"
2,(甜菜),0.2,"[((), (甜菜), 0.2, 1.0)]"
3,(莴苣),0.8,"[((), (莴苣), 0.8, 1.0)]"
4,(葡萄酒),0.6,"[((), (葡萄酒), 0.6, 1.0)]"
5,(豆奶),0.8,"[((), (豆奶), 0.8, 1.0)]"
6,"(橙汁, 尿布)",0.4,"[((), (橙汁, 尿布), 0.4, 1.0), ((尿布), (橙汁), 0.5, 1..."
7,"(甜菜, 尿布)",0.2,"[((), (甜菜, 尿布), 0.2, 1.0), ((尿布), (甜菜), 0.25, ..."
8,"(尿布, 莴苣)",0.6,"[((), (尿布, 莴苣), 0.6, 1.0), ((尿布), (莴苣), 0.7499..."
9,"(尿布, 葡萄酒)",0.6,"[((), (尿布, 葡萄酒), 0.6, 1.0), ((尿布), (葡萄酒), 0.74..."
